In [ ]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

In [ ]:
# зчитуємо дані з таблиці sqlite для прогнозування
engine = create_engine('sqlite:///ts.sqlite3', echo=False)
sqlite_connection = engine.connect()

from sqlalchemy import inspect
inspector = inspect(engine)

# Get table information
print(inspector.get_table_names())

# Get column information
print(inspector.get_columns('ts1'))

df = pd.read_sql_query("SELECT * from ts1", engine)
df

sqlite_connection.close()

In [ ]:
# в таблиці sqlite дані дату у нас в текстовому форматі, тож переформатовуємо в datetime
df['time'] = pd.to_datetime(df['time'])

In [ ]:
# видаляємо зайву колонку index, яка невідомо чому з'явилась
df.drop(columns='index',inplace = True)
df


In [ ]:
# бібліотеки для форекастінгу заватажені без розбору з туторіола
from warnings import simplefilter

import numpy as np
import pandas as pd

from sktime.datasets import load_airline
from sktime.forecasting.arima import ARIMA, AutoARIMA
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.compose import (
    EnsembleForecaster,
    MultiplexForecaster,
    TransformedTargetForecaster,
    make_reduction,
)
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
from sktime.forecasting.model_evaluation import evaluate
from sktime.forecasting.model_selection import (
    ExpandingWindowSplitter,
    ForecastingGridSearchCV,
    SlidingWindowSplitter,
    temporal_train_test_split,
)
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.theta import ThetaForecaster
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.performance_metrics.forecasting import (
    MeanAbsolutePercentageError,
    mean_absolute_percentage_error,
)
from sktime.transformations.series.detrend import Deseasonalizer, Detrender
from sktime.utils.plotting import plot_series
from sktime.forecasting.bats import BATS
from sktime.forecasting.tbats import TBATS

simplefilter("ignore", FutureWarning)
%matplotlib inline

In [ ]:
# зрізаємо нерелеватний період
df = df[df.time.dt.year>2017]

In [ ]:
# перетворюємо дату у формат періоду та підраховуємо кількість подій у цьому періоді
ts1 = df.resample('M',on='time').count()

In [ ]:
# першій та останній період скоріш за все будуть неповними, можливо, це буде впливати на прогноз,можливо,ні, але на графіку
# буде виглядати обрубано, тому їх видаляємо
ts1 = ts1[:-1]
ts1 = ts1[1:]


In [ ]:
# resample ставить дату в індекс, але переформатовує дату на об'єкт,переводимо індекс знову в формат періоду
# та зберігаємо в зміній y
ts1.index = ts1.index.to_period()
y = ts1.time
# для горизонта передбачення краще використати щось типу max_date минус декілька періодів, а не абсолютну дату, ДОРОБИТИ
max_date = max(ts1.index).strftime('%Y-%m-%d')


In [ ]:
fh = ForecastingHorizon(
    pd.PeriodIndex(pd.date_range('2020-09-01', periods=12, freq="M")), is_relative=False
 )

In [ ]:

def prediction_table(y,forecaster,fh,method):
    forecaster.fit(y)
    y_pred = forecaster.predict(fh)
    y_er = y['2020-09-01':]
    max_y = max(y.index).strftime('%Y-%m-%d')
    pred_err = y_pred['2020-09-01':max_y]
    y_pred.index = y_pred.index.to_timestamp()
    pred = pd.DataFrame(y_pred).reset_index()
    pred['method']=method
    pred['MAPE']=mean_absolute_percentage_error(y_er, pred_err)
    pred.columns=[['date','value','method','MAPE']]
    return pred
    
#mean_absolute_percentage_error(y_pred, y)

In [ ]:
forecaster = ExponentialSmoothing(trend="add", seasonal="additive", sp=12)
pred1 = prediction_table(y,forecaster,fh,'ExponentialSmoothing')

In [ ]:
forecaster = AutoARIMA(sp=12, suppress_warnings=True)
pred2 = prediction_table(y,forecaster,fh,'AutoARIMA')

In [ ]:
forecaster = BATS(sp=12, use_trend=True, use_box_cox=True)
pred3 = prediction_table(y,forecaster,fh,'BATS')

In [ ]:
forecaster = EnsembleForecaster([
    ("ses", ExponentialSmoothing(seasonal="multiplicative", sp=12)),
    ("holt", ExponentialSmoothing(trend="add",  seasonal="multiplicative", sp=12)),
    ("damped", ExponentialSmoothing(trend="add",  seasonal="multiplicative", sp=12))
])
pred4 = prediction_table(y,forecaster,fh,'EnsembleForecaster')

In [ ]:
forecaster = TBATS(sp=12, use_trend=True, use_box_cox=True)
pred5 = prediction_table(y,forecaster,fh,'TBATS')

In [ ]:
z = y.copy()
z = z.to_timestamp(freq="M")
fhz = ForecastingHorizon(
    pd.date_range('2020-09-01', periods=12, freq="M"), is_relative=False
 )

In [ ]:
from sktime.forecasting.fbprophet import Prophet
forecaster = Prophet(
    seasonality_mode="multiplicative",
    n_changepoints=int(len(y) / 8),
    add_country_holidays={"country_name": "Ukraine"},
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=True,
)
forecaster.fit(z)
y_pred = forecaster.predict(fhz)

In [ ]:
pr_prophet = pd.DataFrame(y_pred).reset_index()
pr_prophet['ds'] = pd.to_datetime(pr_prophet['ds'])
pr_prophet['method']='Prophet'
z_er = z['2020-09-01':]
max_z = max(z.index).strftime('%Y-%m-%d')
pred_err = y_pred['2020-09-01':max_z]
pr_prophet['MAPE']=mean_absolute_percentage_error(z_er, pred_err)
pr_prophet.columns=[['date','value','method','MAPE']]
pr_prophet

In [ ]:
y.index = y.index.to_timestamp()

fact = pd.DataFrame(y)
fact.rename(columns={'time':'val'},inplace=True)

fact.reset_index(inplace=True)

fact['method']='fact'

fact.columns=[['date','value','method']]

full = pd.concat([fact,pred1,pred2,pred3,pred4,pred5,pr_prophet])

full

In [ ]:
# запис у таблицю
sqlite_connection = engine.connect()
sqlite_table = "ts_pred"
full.to_sql(sqlite_table, sqlite_connection, if_exists='replace')
sqlite_connection.close()